In [ ]:
pip install cryptography stegano pillow scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 43.5 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.1.0
    Uninstalling pillow-11.1.0:
      Successfully uninstalled pillow-11.1.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import base64
import getpass
from cryptography.fernet import Fernet, InvalidToken
from stegano import lsb
from PIL import Image

# Function to create a valid encryption key from input
def create_key_from_input(user_key):
    key_bytes = user_key.encode()
    padded_key = base64.urlsafe_b64encode(key_bytes.ljust(32)[:32])
    return padded_key

# Function to decrypt a message
def decrypt_message(secret_key, encrypted_message):
    cipher_suite = Fernet(secret_key)
    return cipher_suite.decrypt(encrypted_message).decode()

# Function to extract a hidden message from an image
def extract_message(image_path):
    try:
        return lsb.reveal(image_path)
    except Exception:
        return None

# Function to erase the hidden message by resetting the image
def erase_message(image_path):
    try:
        # Reload the image and save it without steganographic content
        with Image.open(image_path) as img:
            img = img.convert("RGB")
            img.save(image_path)
        print(f"Hidden message erased from image: {image_path}")
    except Exception as e:
        print(f"Error erasing the message: {e}")

# Function to decrypt the secure key with a secondary password
def decrypt_secure_key(encrypted_key, image_path):
    print("\nDecrypting the secure key...")
    secondary_password = getpass.getpass("Enter the secondary password to decrypt the key: ").strip()
    if len(secondary_password) < 8:
        print("The password must be at least 8 characters long.")
        erase_message(image_path)
        return None
    encryption_key = create_key_from_input(secondary_password)
    try:
        decrypted_key = decrypt_message(encryption_key, encrypted_key)
        print("Secure key successfully decrypted.")
        return decrypted_key
    except InvalidToken:
        print("Incorrect password. Hidden message will be erased.")
        erase_message(image_path)
        return None

# Main function for extracting and decrypting the secret message
def main():
    # Directory containing images
    images_dir = "/content/drive/MyDrive/New folder"
    image_paths = [os.path.join(images_dir, file) for file in os.listdir(images_dir) if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]

    if len(image_paths) < 1:
        print("No images found in the 'images' directory. Add images to proceed.")
        return

    # Prompt the user to input the encrypted key
    encrypted_key_input = input("Enter the encrypted secure key: ").strip()
    if not encrypted_key_input:
        print("The encrypted key cannot be empty.")
        return

    encrypted_key = encrypted_key_input.encode()

    # Iterate through all images and check for a hidden message
    for image_path in image_paths:
        print(f"\nChecking image: {image_path}")
        hidden_message = extract_message(image_path)
        if hidden_message:
            secure_key = decrypt_secure_key(encrypted_key, image_path)
            if not secure_key:
                return  # Exit if the secure key decryption fails

            try:
                # Attempt to decrypt the hidden message
                decrypted_message = decrypt_message(secure_key.encode(), hidden_message.encode())
                print("\nDecrypted Message Found: ", decrypted_message)
                return
            except InvalidToken:
                print("Incorrect secure key. Unable to decrypt the hidden message.")
                erase_message(image_path)
                return

    print("No hidden message found or incorrect secure key provided.")

if __name__ == "__main__":
    main()


Enter the encrypted secure key: abababab

Checking image: /content/drive/MyDrive/New folder/pushpa.jpg

Checking image: /content/drive/MyDrive/New folder/devara.jpeg

Decrypting the secure key...
Enter the secondary password to decrypt the key: ··········
Incorrect password. Hidden message will be erased.
Hidden message erased from image: /content/drive/MyDrive/New folder/devara.jpeg
